In [1]:
import os
import torch
from transformers import pipeline, AutoTokenizer, GenerationConfig, PhiForCausalLM
from langchain.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载和分割本地知识文档
这里以2024年1月11号发射的[快舟一号甲](https://baike.baidu.com/item/快舟一号甲)的百科词条为例

In [2]:
# 加载本地词向量模型，使用的是 https://huggingface.co/BAAI/bge-base-zh
# model_name = "./data/BAAI_bge-base-zh"
model_name = "BAAI/bge-base-zh"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embedding = HuggingFaceBgeEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs,
                query_instruction="为文本生成向量表示用于文本检索"
            )

In [3]:
model_id = './model_save/dpo/'
tokenizer = AutoTokenizer.from_pretrained(model_id)

def get_token_len(text: str) -> int:
    '''
    统计token长度
    '''
    tokens = tokenizer.encode(text)
    return len(tokens)

In [4]:
doc_db_save_dir = './model_save/vector'

if not os.path.exists(doc_db_save_dir):

    # 1. 从文件读取本地数据集
    loader = TextLoader("./data/快舟一号甲.txt")
    documents = loader.load()

    # 2. 拆分文档
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=96, chunk_overlap=8, length_function=get_token_len,  separators = ["\n\n", "\n", "。", " ", ""])
    splited_documents = text_splitter.split_documents(documents)
    print(splited_documents[0:2])

    # 3. 向量化并保存到本地目录

    db = Chroma.from_documents(splited_documents, embedding, persist_directory=doc_db_save_dir)
    db.persist()
else:
    db = Chroma(persist_directory=doc_db_save_dir,  embedding_function=embedding)

[Document(page_content='快舟一号甲:\n快舟一号甲（英文：Kuaizhou-1A，简称：KZ-1A），是由中国航天科工火箭技术有限公司研制的三级固体运载火箭。', metadata={'source': './data/快舟一号甲.txt'}), Document(page_content='快舟一号甲运载火箭全长约20米，起飞质量约30吨，整流罩最大直径1.4米，太阳同步圆轨道的运载能力为200千克/700千米，近地轨道运载能力为300千克。火箭采用车载机动发射方式，主要面向微小卫星发射和组网，具备一箭多星发射能力。', metadata={'source': './data/快舟一号甲.txt'})]


# 加载对话模型并构造对话prompt

In [4]:

model = PhiForCausalLM.from_pretrained(model_id).to(device)

phi_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16, device=device)

In [5]:
question = "快舟一号甲的近地轨道运载能力是多少？"

In [6]:
# 构造prompt
template = "请根据以下给出的背景知识回答问题，对于不知道的信息，直接回答“未找到相关答案”。\n以下为为背景知识：\n"

similar_docs = db.similarity_search(question, k = 1)
for i, doc in enumerate(similar_docs):
    template += f"{i}. {doc.page_content}"

template += f'\n以下为问题：\n{question}'
print(template)

请根据以下给出的背景知识回答问题，对于不知道的信息，直接回答“未找到相关答案”。
以下为为背景知识：
0. 快舟一号甲:
快舟一号甲（英文：Kuaizhou-1A，简称：KZ-1A），是由中国航天科工火箭技术有限公司研制的三级固体运载火箭。
快舟一号甲运载火箭全长约20米，起飞质量约30吨，整流罩最大直径1.4米，太阳同步圆轨道的运载能力为200千克/700千米，近地轨道运载能力为300千克。火箭采用车载机动发射方式，主要面向微小卫星发射和组网，具备一箭多星发射能力。
2024年1月11日11时52分，中国在酒泉卫星发射中心使用快舟一号甲运载火箭，成功将天行一号02星发射升空，卫星顺利进入预定轨道，发射任务获得圆满成功。
以下为问题：
快舟一号甲的近地轨道运载能力是多少？


In [7]:
prompt = f"##提问:\n{template}\n##回答:\n"
outputs = phi_pipe(prompt, num_return_sequences=1, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)

print(outputs[0]['generated_text'][len(prompt): ])

快艇一号甲的近地轨道运载能力为300千克。
